# Scoring and Citations Testbed
---

## Import Libraries

In [ ]:
import sys 
import os 
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append('/ocean/projects/cis250068p/jhwang4/idl-project')

In [ ]:
import torch as t
import numpy as np
import pandas as pd
import torch.nn.functional as F
from tqdm.auto import tqdm

from transformers import GPTNeoXForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset
from sentence_transformers import SentenceTransformer, util

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.tokenize import sent_tokenize

from sentence_transformers import SentenceTransformer
from rouge_score import rouge_scorer
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import re
from scipy.spatial.distance import cosine


In [ ]:
from src.model import load_fo_model, load_ba_model, DEVICE
from src.data import load_cnn_dataset
#from src.utils import *
from src.utils_batch_v2 import *
#from src.search import *
from src.search_batch_v2 import *

In [ ]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.float_format', '{:.4f}'.format)

In [ ]:
nltk.download("punkt_tab")

In [ ]:
DEVICE

## Quick Test

In [ ]:
# Load the models
fo_model, fo_tokenizer = load_fo_model()
ba_model, ba_tokenizer = load_ba_model()
fo_model = fo_model.to(DEVICE)
ba_model = ba_model.to(DEVICE)
fo_model.half()
ba_model.half()
if t.cuda.device_count() > 1:
    print(t.cuda.device_count())
    fo_model = t.nn.DataParallel(fo_model)
    ba_model = t.nn.DataParallel(ba_model)

# Example Text
# article = "Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him."
# summary = "Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday"
# adverse_summary = "Daniel Craig is recasted as James Bond again"

In [ ]:
# debug = True

# # In normal, query is sentence/article, and answer is summary/highlight (S->A direction)
# base = calculate_score(summary, article, fo_model, fo_tokenizer, backward=False, query_direction="normal", debug=debug)

# # In Fo, query is summary/highlight , and answer is sentence/article(A->S direction)
# fo = calculate_score(summary, article, fo_model, fo_tokenizer, backward=False, query_direction="reverse", debug=debug)

# # In Ba, query is summary/highlight , and answer is sentence/article(A->S direction)
# ba = calculate_score(summary, article, ba_model, ba_tokenizer, backward=True, query_direction="reverse", debug=debug)


# adv_base = calculate_score(adverse_summary, article, fo_model, fo_tokenizer, backward=False, query_direction="normal", debug=debug)

# adv_fo = calculate_score(adverse_summary, article, fo_model, fo_tokenizer, backward=False, query_direction="reverse", debug=debug)

# adv_ba = calculate_score(adverse_summary, article, ba_model, ba_tokenizer, backward=True, query_direction="reverse", debug=debug)


# scores_data = {
#     'Model Type': ['Base', 'Forward', 'Backward', 'Adv Base', 'Adv Forward', 'Adv Backward'],
    
#     'Query Direction': ['S->A', 'A->S', 'A->S', 'S->A', 'A->S', 'A->S'],
    
#     'Sequence Log Prob': [
#         base['sequence_log_prob'],
#         fo['sequence_log_prob'],
#         ba['sequence_log_prob'],
        
#         adv_base['sequence_log_prob'],
#         adv_fo['sequence_log_prob'],
#         adv_ba['sequence_log_prob'],
#     ],
#     'Normalized Log Prob': [
#         base['normalized_log_prob'],
#         fo['normalized_log_prob'],
#         ba['normalized_log_prob'],
        
#         adv_base['normalized_log_prob'],
#         adv_fo['normalized_log_prob'],
#         adv_ba['normalized_log_prob'],
#     ],
#     'Perplexity': [
#         base['perplexity'],
#         fo['perplexity'],
#         ba['perplexity'],
        
#         adv_base['perplexity'],
#         adv_fo['perplexity'],
#         adv_ba['perplexity'],
#     ]
# }

In [ ]:
# result_df = pd.DataFrame(scores_data)
# result_df

## Linear Search

In [ ]:
dataset = load_cnn_dataset(num_samples=47852)
#dataset = load_cnn_dataset()
dataset = pd.DataFrame(dataset)

In [ ]:
dataset.head(5)

In [ ]:
print(dataset.shape[0])

In [ ]:
#linear_results = linear_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer) # 50 - 59.4s
linear_results = linear_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer, sentence_batch_size=500) # 52.4s, 2m30.9s, 6h18m49s

In [ ]:
#binary_results = binary_search_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer) # 50 - 25.7s
binary_results = binary_search_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer, max_iterations=30, sentence_batch_size=1000) # 35.7s, 7m 47.6s, 5m 15.6s, 5m0.8s

In [ ]:
t.cuda.empty_cache()

In [ ]:
#exclusion_results = exclusion_search_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer) # 50 - 4m54.5s
exclusion_results = exclusion_search_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer, sentence_batch_size=32) # 3m30.4s

In [ ]:
# def calculate_embedding_similarity(highlight, citation):
#     """Calculate cosine similarity between sentence embeddings."""
#     model = SentenceTransformer('all-MiniLM-L6-v2')
    
#     # Generate embeddings
#     highlight_embedding = model.encode([highlight])[0]
#     citation_embedding = model.encode([citation])[0]
    
#     # Calculate cosine similarity (1 - cosine distance)
#     similarity = 1 - cosine(highlight_embedding, citation_embedding)
#     return similarity

# def calculate_rouge_score(highlight, citation):
#     """Calculate ROUGE-L F-measure score."""
#     scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
#     scores = scorer.score(highlight, citation)
#     return scores['rougeL'].fmeasure

# def calculate_tfidf_score(highlight, citation):
#     """Calculate TF-IDF similarity score."""
#     vectorizer = TfidfVectorizer()
#     tfidf_matrix = vectorizer.fit_transform([highlight, citation])
    
#     # Convert sparse matrix to dense array for cosine similarity calculation
#     dense_matrix = tfidf_matrix.toarray()
    
#     # Calculate cosine similarity
#     similarity = 1 - cosine(dense_matrix[0], dense_matrix[1])
#     return similarity

# def process_data(data):
#     """Process the data and reformat to show scores by citation type in columns."""
#     results = []
    
#     for item in data:
#         highlight = item['highlight']
#         result_entry = {'id': item['id'], 'highlight': highlight}
        
#         # Process each citation type
#         for citation_type in ['base_citation', 'ba_citation', 'fo_citation']:
#             prefix = citation_type.split('_')[0]  # Extract 'base', 'ba', or 'fo'
            
#             if citation_type in item and item[citation_type]:
#                 citation = item[citation_type]
                
#                 # Calculate scores
#                 emb_similarity = calculate_embedding_similarity(highlight, citation)
#                 rouge_score = calculate_rouge_score(highlight, citation)
#                 tfidf_score = calculate_tfidf_score(highlight, citation)
                
#                 # Add scores as columns with prefix
#                 result_entry[f'{prefix}_emb_similarity'] = emb_similarity
#                 result_entry[f'{prefix}_rouge_score'] = rouge_score
#                 result_entry[f'{prefix}_tfidf_score'] = tfidf_score
                
#             else:
#                 # Set default values if citation doesn't exist
#                 result_entry[f'{prefix}_emb_similarity'] = None
#                 result_entry[f'{prefix}_rouge_score'] = None
#                 result_entry[f'{prefix}_tfidf_score'] = None
                
#         results.append(result_entry)
    
#     return results

# # Process the data
# linear_final_results = process_data(linear_results)
# binary_final_results = process_data(binary_results)
# #exclusion_final_results = process_data(exclusion_results)


In [ ]:
EMB_MODEL    = SentenceTransformer('all-MiniLM-L6-v2')
ROUGE_SCORER = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
TFIDF_VEC    = TfidfVectorizer(
    token_pattern=r"(?u)\b\w+\b",  
    stop_words=None               
)

def calculate_embedding_similarity(highlight, citation):
    he = EMB_MODEL.encode([highlight])[0]
    ce = EMB_MODEL.encode([citation])[0]
    return 1 - cosine(he, ce)

def calculate_rouge_score(highlight, citation):
    return ROUGE_SCORER.score(highlight, citation)['rougeL'].fmeasure

def calculate_tfidf_score(highlight, citation):
    try:
        mat = TFIDF_VEC.fit_transform([highlight, citation]).toarray()
        return 1 - cosine(mat[0], mat[1])
    except ValueError:
        return 0.0

def process_data(data):
    results = []
    for item in data:
        hl = item['highlight']
        rec = {'id': item['id'], 'highlight': hl}
        for citation_type in ['base_citation','fo_citation','ba_citation']:
            prefix = citation_type.split('_')[0]
            cit = item.get(citation_type, None)
            if cit:
                rec[f'{prefix}_emb_similarity'] = calculate_embedding_similarity(hl, cit)
                rec[f'{prefix}_rouge_score']     = calculate_rouge_score(hl, cit)
                rec[f'{prefix}_tfidf_score']     = calculate_tfidf_score(hl, cit)
            else:
                rec[f'{prefix}_emb_similarity'] = None
                rec[f'{prefix}_rouge_score']     = None
                rec[f'{prefix}_tfidf_score']     = None
        results.append(rec)
    return results

#linear_final_results    = process_data(linear_results)
#binary_final_results    = process_data(binary_results)
exclusion_final_results = process_data(exclusion_results)

In [ ]:
r = pd.DataFrame(linear_final_results) 
r.drop(['id', 'highlight'], axis=1).mean()
r.to_csv("/ocean/projects/cis250068p/jhwang4/idl-project/notebooks/linear_result_500.csv", index=True)

In [ ]:
r = pd.DataFrame(binary_final_results) 
r.drop(['id', 'highlight'], axis=1).mean()
#r.to_csv("binary_500_results.csv", index=True)

In [ ]:
r = pd.DataFrame(exclusion_final_results) 
r.drop(['id', 'highlight'], axis=1).mean()

In [ ]:
df_linear = pd.DataFrame(linear_final_results)
df_binary = pd.DataFrame(binary_final_results)
df_exclusion = pd.DataFrame(exclusion_final_results)

df_merged = pd.merge(df_linear, df_binary, on=['id', 'highlight'],suffixes=('_linear', '_binary'))

df_exclusion = df_exclusion.add_suffix('_exclusion')
df_exclusion = df_exclusion.rename(columns={'id_exclusion': 'id', 'highlight_exclusion': 'highlight'})

df_merged = pd.merge(df_merged, df_exclusion, on=['id', 'highlight'])

mean_series = df_merged.drop(['id', 'highlight'], axis=1).mean()

data = {
    'Base_linear': [
        mean_series['base_emb_similarity_linear'],
        mean_series['base_rouge_score_linear'],
        mean_series['base_tfidf_score_linear']
    ],
    'Fo_linear': [
        mean_series['fo_emb_similarity_linear'],
        mean_series['fo_rouge_score_linear'],
        mean_series['fo_tfidf_score_linear']
    ],
    'Ba_linear': [
        mean_series['ba_emb_similarity_linear'],
        mean_series['ba_rouge_score_linear'],
        mean_series['ba_tfidf_score_linear']
    ],
    'Base_binary': [
        mean_series['base_emb_similarity_binary'],
        mean_series['base_rouge_score_binary'],
        mean_series['base_tfidf_score_binary']
    ],
    'Fo_binary': [
        mean_series['fo_emb_similarity_binary'],
        mean_series['fo_rouge_score_binary'],
        mean_series['fo_tfidf_score_binary']
    ],
    'Ba_binary': [
        mean_series['ba_emb_similarity_binary'],
        mean_series['ba_rouge_score_binary'],
        mean_series['ba_tfidf_score_binary']
    ],
    'Base_exclusion': [
        mean_series['base_emb_similarity_exclusion'],
        mean_series['base_rouge_score_exclusion'],
        mean_series['base_tfidf_score_exclusion']
    ],
    'Fo_exclusion': [
        mean_series['fo_emb_similarity_exclusion'],
        mean_series['fo_rouge_score_exclusion'],
        mean_series['fo_tfidf_score_exclusion']
    ],
    'Ba_exclusion': [
        mean_series['ba_emb_similarity_exclusion'],
        mean_series['ba_rouge_score_exclusion'],
        mean_series['ba_tfidf_score_exclusion']
    ],
}

table_df = pd.DataFrame(data, index=['Embedding', 'Rouge', 'Tfidf'])
table_df

In [ ]:
df_linear = pd.DataFrame(linear_final_results)
mean_series = df_linear.drop(['id', 'highlight'], axis=1).mean()

data = {
    'Base_linear': [
        mean_series['base_emb_similarity'],
        mean_series['base_rouge_score'],
        mean_series['base_tfidf_score']
    ],
    'Fo_linear': [
        mean_series['fo_emb_similarity'],
        mean_series['fo_rouge_score'],
        mean_series['fo_tfidf_score']
    ],
    'Ba_linear': [
        mean_series['ba_emb_similarity'],
        mean_series['ba_rouge_score'],
        mean_series['ba_tfidf_score']
    ]
}

table_df = pd.DataFrame(data, index=['Embedding', 'Rouge', 'Tfidf'])
table_df.to_csv("/ocean/projects/cis250068p/jhwang4/idl-project/notebooks/linear_result.csv", index=True)

In [ ]:
table_df

In [ ]:
table_df.to_csv("/ocean/projects/cis250068p/jhwang4/idl-project/notebooks/linear_result.csv", index=True)